In [32]:
## Import dependencies 

import pandas as pd

In [33]:
## Read CSV file into Pandas DataFrame from URL 
## The target URL is the ASX company directory, this data will be scraped and stored in a PostgreSQL database 
## Creating a web-scraping programme will enable the database to be updated automatically when new companies are added to the index 

# Create URL object, assigning the target URL 

URL = "https://asx.api.markitdigital.com/asx-research/1.0/companies/directory/file?access_token=83ff96335c2d45a094df02a206a39ff4"

# Create data object to store results, use pd.read_csv to return DataFrame 

data = pd.read_csv(URL)    

# Print results 

data

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D,1414 DEGREES LIMITED,Capital Goods,12/09/2018,10685244
1,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,8800309
2,1AE,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,10874088
3,1AG,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,6965525
4,1CG,ONE CLICK GROUP LIMITED,Capital Goods,28/04/2017,5487227
...,...,...,...,...,...
2053,ZLD,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,17474619
2054,ZMI,ZINC OF IRELAND NL,Materials,18/09/2007,4476030
2055,ZMM,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,4153301
2056,ZNC,ZENITH MINERALS LIMITED,Materials,29/05/2007,34533327


In [34]:
## Import dependencies 
# yahooquery library will be used to access quotes 

from yahooquery import Ticker

In [35]:
# Add the .AX suffix to the ASX code, this is required to exectue the yahooquery search

data['ASX code'] = data['ASX code'].astype(str) + '.AX'  

# Print results 

data

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D.AX,1414 DEGREES LIMITED,Capital Goods,12/09/2018,10685244
1,1AD.AX,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,8800309
2,1AE.AX,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,10874088
3,1AG.AX,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,6965525
4,1CG.AX,ONE CLICK GROUP LIMITED,Capital Goods,28/04/2017,5487227
...,...,...,...,...,...
2053,ZLD.AX,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,17474619
2054,ZMI.AX,ZINC OF IRELAND NL,Materials,18/09/2007,4476030
2055,ZMM.AX,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,4153301
2056,ZNC.AX,ZENITH MINERALS LIMITED,Materials,29/05/2007,34533327


In [36]:
## Create Ticker object with each ticker in the ASX code column 

ticker = Ticker(data['ASX code'], asynchronous=True)

# Request price history for each ticker, over a period of 1 year 

stockPrice = ticker.history(period="1y").reset_index()

# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,adjclose,splits,dividends
0,14D.AX,2022-06-23,0.078,0.079,0.077,0.079,193256.0,0.079,0.0,0.0
1,14D.AX,2022-06-24,0.080,0.081,0.079,0.081,84272.0,0.081,0.0,0.0
2,14D.AX,2022-06-27,0.081,0.081,0.077,0.080,26417.0,0.080,0.0,0.0
3,14D.AX,2022-06-28,0.077,0.079,0.077,0.079,41106.0,0.079,0.0,0.0
4,14D.AX,2022-06-29,0.077,0.077,0.077,0.077,33050.0,0.077,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
505504,ZNO.AX,2023-06-19,0.044,0.044,0.044,0.044,8013.0,0.044,0.0,0.0
505505,ZNO.AX,2023-06-20,0.044,0.044,0.042,0.042,383066.0,0.042,0.0,0.0
505506,ZNO.AX,2023-06-21,0.042,0.042,0.041,0.041,60587.0,0.041,0.0,0.0
505507,ZNO.AX,2023-06-22,0.041,0.041,0.041,0.041,9046.0,0.041,0.0,0.0


In [37]:
## Delete the DataFrame columns that are not required

stockPrice.drop(['splits', 'dividends', 'adjclose'], axis=1, inplace=True)

In [38]:
## Calculate the percentage change, based on closing price of the market, by using the pct_change method

stockPrice['percentage change'] = (stockPrice['close'] - stockPrice['open']) / stockPrice['open']

# Round the numerical data to three decimal places 

stockPrice = stockPrice.round(3)

# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,percentage change
0,14D.AX,2022-06-23,0.078,0.079,0.077,0.079,193256.0,0.013
1,14D.AX,2022-06-24,0.080,0.081,0.079,0.081,84272.0,0.013
2,14D.AX,2022-06-27,0.081,0.081,0.077,0.080,26417.0,-0.012
3,14D.AX,2022-06-28,0.077,0.079,0.077,0.079,41106.0,0.026
4,14D.AX,2022-06-29,0.077,0.077,0.077,0.077,33050.0,0.000
...,...,...,...,...,...,...,...,...
505504,ZNO.AX,2023-06-19,0.044,0.044,0.044,0.044,8013.0,0.000
505505,ZNO.AX,2023-06-20,0.044,0.044,0.042,0.042,383066.0,-0.045
505506,ZNO.AX,2023-06-21,0.042,0.042,0.041,0.041,60587.0,-0.024
505507,ZNO.AX,2023-06-22,0.041,0.041,0.041,0.041,9046.0,0.000


In [39]:
# Create function to assign gain, loss or no change to stock price movement based on percentage change value

def f(stockPrice):
    if stockPrice['percentage change'] < 0:
        val = "Loss"
    elif stockPrice['percentage change'] > 0:
        val = "Gain"
    else:
        val = "No Change"
    return val

In [40]:
# Create new column titled gain/loss and apply function created above to assign appropriate values 

stockPrice['gain/loss'] = stockPrice.apply(f, axis=1)

In [41]:
# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,percentage change,gain/loss
0,14D.AX,2022-06-23,0.078,0.079,0.077,0.079,193256.0,0.013,Gain
1,14D.AX,2022-06-24,0.080,0.081,0.079,0.081,84272.0,0.013,Gain
2,14D.AX,2022-06-27,0.081,0.081,0.077,0.080,26417.0,-0.012,Loss
3,14D.AX,2022-06-28,0.077,0.079,0.077,0.079,41106.0,0.026,Gain
4,14D.AX,2022-06-29,0.077,0.077,0.077,0.077,33050.0,0.000,No Change
...,...,...,...,...,...,...,...,...,...
505504,ZNO.AX,2023-06-19,0.044,0.044,0.044,0.044,8013.0,0.000,No Change
505505,ZNO.AX,2023-06-20,0.044,0.044,0.042,0.042,383066.0,-0.045,Loss
505506,ZNO.AX,2023-06-21,0.042,0.042,0.041,0.041,60587.0,-0.024,Loss
505507,ZNO.AX,2023-06-22,0.041,0.041,0.041,0.041,9046.0,0.000,No Change


In [42]:
stockPrice.to_csv('Desktop/stockprice.csv')